In [44]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/KaggleV2-May-2016.csv")
df.columns = df.columns.str.lower().str.replace('-', '_')

df.head()

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [45]:
df['scheduledday'] = pd.to_datetime(df['scheduledday'])
df['appointmentday'] = pd.to_datetime(df['appointmentday'])

In [46]:
df['scheduled_date'] = df['scheduledday'].dt.date
df['scheduled_time'] = df['scheduledday'].dt.hour

df['appointment_date'] = df['appointmentday'].dt.date
df['appointment_day_of_week'] = df['appointmentday'].dt.dayofweek
df['is_weekend'] = df['appointment_day_of_week'].isin([5,6]).astype(int)

In [47]:
df['days_wait'] = (df['appointmentday'] - df['scheduledday']).dt.days

In [48]:
df['age'] = df['age'].apply(lambda x: 0 if x < 0 else x)

In [49]:
df = df.rename(columns={
    'hipertension': 'hypertension',
    'handcap': 'handicap'
})

In [50]:
df['patientid'] = df['patientid'].astype(str).str.split('.').str[0]

In [51]:
df['no_show_binary'] = df['no_show'].map({'No': 0, 'Yes': 1})

# Make sure no_show_binary exists
df['no_show_binary'] = df['no_show'].map({'No': 0, 'Yes': 1})

# Sort by patient and time so "past" is really past
df = df.sort_values(['patientid', 'scheduledday'])

# Number of appointments *before* this one
df['patient_total_appointments'] = df.groupby('patientid').cumcount()

# Cumulative past no-shows, shifted so current row's label is not included
df['patient_past_no_shows'] = (
    df.groupby('patientid')['no_show_binary']
      .cumsum()
      .shift(fill_value=0)
)

# No-show ratio based only on past appointments
df['patient_no_show_ratio'] = np.where(
    df['patient_total_appointments'] > 0,
    df['patient_past_no_shows'] / df['patient_total_appointments'],
    0
)

In [52]:
df['target'] = df['no_show'].map({'No': 0, 'Yes': 1})

In [53]:
df.to_csv("../data/processed/processed_data.csv", index=False)

In [54]:
df.shape

(110527, 25)

In [55]:
df.to_csv("../data/processed/processed_data.csv", index=False)